# Enron emails

Import files

In [2]:
# from kaggle.api.kaggle_api_extended import KaggleApi
# api = KaggleApi()
# api.authenticate()
# api.dataset_download_files("wcukierski/enron-email-dataset")

In [3]:
import pandas as pd
import numpy as np
import pymongo

In [11]:
emails_df = pd.read_csv("emails.csv", nrows=10000)

In [12]:
print(emails_df.shape)
emails_df.head(3)

(10000, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...


In [131]:
print(emails_df['message'][63])

Message-ID: <1776521.1075855688576.JavaMail.evans@thyme>
Date: Wed, 6 Sep 2000 04:46:00 -0700 (PDT)
From: phillip.allen@enron.com
To: thomas.martin@enron.com, mike.grigsby@enron.com, keith.holst@enron.com, 
	jay.reitmeyer@enron.com, frank.ermis@enron.com
Subject: Wow
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Thomas A Martin, Mike Grigsby, Keith Holst, Jay Reitmeyer, Frank Ermis
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/06/2000 
10:49 AM ---------------------------


Jeff Richter
09/06/2000 07:39 AM
To: Phillip K Allen/HOU/ECT@ECT
cc:  
Subject: Wow


---------------------- Forwarded by Jeff Richter/HOU/ECT on 09/06/2000 09:45 
AM ---------------------------
To: Mike Swerzbin/HOU/ECT@ECT, Robert Badeer/HOU/ECT@ECT, Sean 
Crandall/PDX/ECT@ECT, Tim Belden/HOU/ECT@ECT, Jeff

In [18]:
emails_df.drop_duplicates(subset="message", inplace=True)
emails_df.dropna(axis=1, inplace=True)
print(emails_df.shape)

(10000, 2)


### A few figures

In [46]:
print("Number of 'na' values' in dataset messages:")
print(pd.isna(emails_df["message"]).sum())

print("Number of duplicates in dataset messages:")
print(emails_df.duplicated(subset="message").sum())

print("Average email length (medata included)")
print(emails_df["message"].apply(len).mean())
print("Median email length (medata included)")
print(emails_df["message"].apply(len).median())


Number of 'na' values' in dataset messages:
0
Number of duplicates in dataset messages:
0
Average email length (medata included)
2363.287
Median email length (medata included)
1164.0


Pas de doublon ni du "null" a priori

## Let's put all this into a MongoDB database!

In [83]:
client = pymongo.MongoClient(host="127.0.0.1",
                     port=27017)

db = client["simplon"]
col = db["raw_mail"]

In [107]:
# make a Python list with all items
mongo_docs = [{"filename": emails_df["file"].iloc[row],
               "message": emails_df["message"].iloc[row]} for row in range(emails_df.shape[0])]

# mongo_docs[0]

In [108]:
# insert the list of documents into the Mongo DB
result = col.insert_many(mongo_docs)
print(len(result.inserted_ids), "inserted documents.")
print("An example:", col.find_one())

10000 inserted documents.
An example: {'_id': ObjectId('5e831695c3a1888d5f8650fa'), 'filename': 'allen-p/_sent_mail/1.', 'message': "Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "}


## A little of cleaning

(We do the cleaning on the dataframe)

### Pure Python - didn't work

In [133]:
def get_text(row_idx):
    '''Returns the content from an email string object'''
    filename = emails_df["file"].iloc[row_idx]
    msg = emails_df["message"].iloc[row_idx]
    msg_list = msg.split("\n")
    dic = {"filename": filename}
    
    for row in range(len(msg_list)):
        str_row = msg_list[row]

        if str_row != "":
            sep = str_row.find(": ")
            k = str_row[:sep]
            v = str_row[sep+2 :]
            if v != "":
                dic[k] = v
        else:
            dic["body"] = "\n".join(msg_list[row + 1:-1])
            break
    return dic

# test example
#print(get_text(0).keys())

# Check if all the format is correct
def keys_ok(any_dic):
    for key in any_dic.keys():
        if key.find(".") != -1:
#            print(any_dic)
            return False
    return True

all([keys_ok(get_text(i)) for i in range(emails_df.shape[0])])

False